## New Distance Metrics for Probability Distribution and Bag of Words 

A small tutorial to illustrate the new distance functions.

We would need this mostly when comparing how similar two probability distributions are, and in the case of gensim, usually for LSI or LDA topic distributions after we have a LDA model.

Gensim already has functionalities for this, in the sense of getting most similar documents - [this](http://radimrehurek.com/topic_modeling_tutorial/3%20-%20Indexing%20and%20Retrieval.html), [this](https://radimrehurek.com/gensim/tut3.html) and [this](https://radimrehurek.com/gensim/similarities/docsim.html) are such examples of documentation and tutorials.

What this tutorial shows is a building block of these larger methods, which are a small suite of distance metrics.
We'll start by setting up a small corpus and showing off the methods.


一个小教程来说明新的距离函数。

在比较两个概率分布的相似程度时，我们最需要这样做，在gensim的情况下，通常是在我们有了LDA模型之后，针对LSI或LDA主题分布。

Gensim已经具备了这方面的功能，在获取大多数类似文档的意义上——这、这和这都是文档和教程的示例。

本教程展示的是这些较大方法的构建块，这些方法是一小套距离度量。我们将首先建立一个小的语料库并展示这些方法。

In [3]:
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
import numpy

In [4]:
# you can use any corpus, this is just illustratory

texts = [['bank','river','shore','water'],
        ['river','water','flow','fast','tree'],
        ['bank','water','fall','flow'],
        ['bank','bank','water','rain','river'],
        ['river','water','mud','tree'],
        ['money','transaction','bank','finance'],
        ['bank','borrow','money'], 
        ['bank','finance'],
        ['finance','money','sell','bank'],
        ['borrow','sell'],
        ['bank','loan','sell']]

dictionary = Dictionary(texts)
print("字典：",dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
print("语料库：",corpus)

字典： Dictionary(16 unique tokens: ['bank', 'river', 'shore', 'water', 'fast']...)
语料库： [[(0, 1), (1, 1), (2, 1), (3, 1)], [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(0, 1), (3, 1), (5, 1), (7, 1)], [(0, 2), (1, 1), (3, 1), (8, 1)], [(1, 1), (3, 1), (6, 1), (9, 1)], [(0, 1), (10, 1), (11, 1), (12, 1)], [(0, 1), (11, 1), (13, 1)], [(0, 1), (10, 1)], [(0, 1), (10, 1), (11, 1), (14, 1)], [(13, 1), (14, 1)], [(0, 1), (14, 1), (15, 1)]]


In [5]:
#numpy.random.seed(1) # setting random seed to get the same results each time.
model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=2)

model.show_topics()

[(0,
  '0.158*"bank" + 0.106*"water" + 0.105*"river" + 0.091*"money" + 0.068*"tree" + 0.068*"sell" + 0.065*"finance" + 0.046*"flow" + 0.045*"borrow" + 0.042*"mud"'),
 (1,
  '0.206*"bank" + 0.108*"water" + 0.081*"finance" + 0.076*"sell" + 0.068*"river" + 0.066*"borrow" + 0.064*"flow" + 0.059*"fall" + 0.053*"rain" + 0.044*"money"')]

Let's take a few sample documents and get them ready to test Similarity. Let's call the 1st topic the water topic and the second topic the finance topic.

Note: these are all distance metrics. This means that a value between 0 and 1 is returned, where values closer to 0 indicate a smaller 'distance' and therefore a larger similarity.

In [6]:
doc_water = ['river', 'water', 'shore']
doc_finance = ['finance', 'money', 'sell']
doc_bank = ['finance', 'bank', 'tree', 'water']

# now let's make these into a bag of words format
##实现CBOW的方法
bow_water = model.id2word.doc2bow(doc_water)   
print("bow_water：",bow_water)
bow_finance = model.id2word.doc2bow(doc_finance)   
print("bow_finance:",bow_finance)
bow_bank = model.id2word.doc2bow(doc_bank)   
print("bow_bank:",bow_bank)
#print("2：",model.id2word.doc2bow(["bank"]))
# we can now get the LDA topic distributions for these
lda_bow_water = model[bow_water]
print("lda_bow_water:",lda_bow_water)
lda_bow_finance = model[bow_finance]
print("lda_bow_finance:",lda_bow_finance)
lda_bow_bank = model[bow_bank]
print("lda_bow_bank：",lda_bow_bank)
print("finance-water:",((0.8548162-0.14528602)**2 + (0.14518377-0.854714)**2)**0.5)
print("finance-bank:",((0.8548162-0.40596375)**2 + (0.14518377-0.5940363)**2)**0.5)

bow_water： [(1, 1), (2, 1), (3, 1)]
bow_finance: [(10, 1), (11, 1), (14, 1)]
bow_bank: [(0, 1), (3, 1), (6, 1), (10, 1)]
lda_bow_water: [(0, 0.8328832), (1, 0.16711684)]
lda_bow_finance: [(0, 0.7771238), (1, 0.22287622)]
lda_bow_bank： [(0, 0.7782738), (1, 0.22172624)]
finance-water: 1.003427238824363
finance-bank: 0.6347732788629366


## Jaccard

In [5]:
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
print("1:",jaccard(lda_bow_water, lda_bow_finance))
print("2:",jaccard(lda_bow_finance, lda_bow_bank))
#print("3:",jaccard(lda_bow_bank, lda_bow_water))

1: 0.8597928477777432
2: 0.8397793894973307


## Hellinger and Kullback–Leibler

We're now ready to apply our distance metrics.

Let's start with the popular Hellinger distance. 
The Hellinger distance metric gives an output in the range [0,1] for two probability distributions, with values closer to 0 meaning they are more similar.

In [6]:
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
hellinger(lda_bow_water, lda_bow_finance)

0.5528494030449027

In [7]:
hellinger(lda_bow_finance, lda_bow_bank)

0.5167124677493228

In [8]:
hellinger(lda_bow_bank, lda_bow_water)

0.03903241214774338

Makes sense, right? In the first example, Document 1 and Document 2 are hardly similar, so we get a value of roughly 0.5. 

In the second case, the documents are a lot more similar, semantically. Trained with the model, they give a much less distance value.

Let's run similar examples down with Kullback Leibler.

In [9]:
kullback_leibler(lda_bow_water, lda_bow_bank)

0.0058879964

In [10]:
kullback_leibler(lda_bow_finance, lda_bow_bank)

1.100847

*NOTE!*

KL is not a Distance Metric in the mathematical sense, and hence is not symmetrical. 
This means that `kullback_leibler(lda_bow_finance, lda_bow_bank)` is not equal to  `kullback_leibler(lda_bow_bank, lda_bow_finance)`. 

In [11]:
# As you can see, the values are not equal. We'll get more into the details of this later on in the notebook.
kullback_leibler(lda_bow_bank, lda_bow_finance)

1.1573412

In our previous examples we saw that there were lower distance values between bank and finance than for bank and water, even if it wasn't by a huge margin. What does this mean?

The `bank` document is a combination of both water and finance related terms - but as bank in this context is likely to belong to the finance topic, the distance values are less between the finance and bank bows.

In [12]:
# just to confirm our suspicion that the bank bow is more to do with finance:

model.get_document_topics(bow_bank)

[(0, 0.8236426), (1, 0.1763574)]

It's evident that while it isn't too skewed, it it more towards the finance topic.

Distance metrics (also referred to as similarity metrics), as suggested in the examples above, are mainly for probability distributions, but the methods can accept a bunch of formats for input. You can do some further reading on [Kullback Leibler](https://en.wikipedia.org/wiki/Kullback–Leibler_divergence) and [Hellinger](https://en.wikipedia.org/wiki/Hellinger_distance) to figure out what suits your needs.

## Jaccard 

Let us now look at the [Jaccard Distance](https://en.wikipedia.org/wiki/Jaccard_index) metric for similarity between bags of words (i.e, documents)

In [13]:
print("bow_water:",bow_water)
jaccard(bow_water, bow_bank)

bow_water: [(1, 1), (2, 1), (3, 1)]


0.8571428571428572

In [14]:
print("doc_water：",doc_water)
jaccard(doc_water, doc_bank)

doc_water： ['river', 'water', 'shore']


0.8333333333333334

In [15]:
jaccard(['word'], ['word'])

0.0

In [16]:
jaccard(['word','word1'], ['word','word2'])

0.6666666666666667

The three examples above feature 2 different input methods. 

In the first case, we present to jaccard document vectors already in bag of words format. The distance can be defined as 1 minus the size of the intersection upon the size of the union of the vectors. 

We can see (on manual inspection as well), that the distance is likely to be high - and it is. 

The last two examples illustrate the ability for jaccard to accept even lists (i.e, documents) as inputs.
In the last case, because they are the same vectors, the value returned is 0 - this means the distance is 0 and they are very similar. 

## Distance Metrics for Topic Distributions

While there are already standard methods to identify similarity of documents, our distance metrics has one more interesting use-case: topic distributions. 

Let's say we want to find out how similar our two topics are, water and finance.

In [17]:
topic_water, topic_finance = model.show_topics()
#print("topic_water:",topic_water)
# some pre processing to get the topics in a format acceptable to our distance metrics

def make_topics_bow(topic):
    # takes the string returned by model.show_topics()
    # split on strings to get topics and the probabilities
    topic = topic.split('+')
    #print("topic:",topic)
    # list to store topic bows
    topic_bow = []
    for word in topic:
        # split probability and word
        prob, word = word.split('*')
        # get rid of spaces
        word = word.replace(" ","")
        #print("word:",word)
        # convert to word_type
        #print("2：",model.id2word.doc2bow(["bank"]))
        #print("example:",model.id2word.doc2bow([word.replace('"','')]))
        word = model.id2word.doc2bow([word.replace('"','')])[0][0]
        topic_bow.append((word, float(prob)))
    return topic_bow
print("topic_finance[1]:",topic_finance[1])
finance_distribution = make_topics_bow(topic_finance[1])
print("finance_distribution:",finance_distribution)
water_distribution = make_topics_bow(topic_water[1])
print("water_distribution:",water_distribution)

# the finance topic in bag of words format looks like this:
#finance_distribution

topic_finance[1]: 0.185*"bank" + 0.113*"money" + 0.103*"sell" + 0.093*"finance" + 0.068*"borrow" + 0.058*"water" + 0.056*"transaction" + 0.053*"loan" + 0.050*"fall" + 0.047*"flow"
finance_distribution: [(0, 0.185), (11, 0.113), (14, 0.103), (10, 0.093), (13, 0.068), (3, 0.058), (12, 0.056), (15, 0.053), (7, 0.05), (5, 0.047)]
water_distribution: [(0, 0.173), (3, 0.146), (1, 0.131), (6, 0.069), (5, 0.058), (10, 0.054), (14, 0.047), (8, 0.046), (2, 0.044), (4, 0.043)]


Now that we've got our topics in a format more acceptable by our functions, let's use a Distance metric to see how similar the word distributions in the topics are.

In [18]:
hellinger(water_distribution, finance_distribution)

0.5957552974303146

Our value of roughly 0.36 means that the topics are not TOO distant with respect to their word distributions.
This makes sense again, because of overlapping words like `bank` and a small size dictionary.

## Some things to take care of 

In our previous example we didn't use Kullback Leibler to test for similarity for a reason - KL is not a Distance 'Metric' in the technical sense (you can see what a metric is [here](https://en.wikipedia.org/wiki/Metric_(mathematics)). The nature of it, mathematically also means we must be a little careful before using it, because since it involves the log function, a zero can mess things up. For example:

In [19]:

# 16 here is the number of features the probability distribution draws from
kullback_leibler(water_distribution, finance_distribution, 16) 

inf

That wasn't very helpful, right? This just means that we have to be a bit careful about our inputs. Our old example didn't work out because they were some missing values for some words (because `show_topics()` only returned the top 10 topics). 

This can be remedied, though.

In [20]:
# return ALL the words in the dictionary for the topic-word distribution.
topic_water, topic_finance = model.show_topics(num_words=len(model.id2word))

# do our bag of words transformation again
finance_distribution = make_topics_bow(topic_finance[1])
water_distribution = make_topics_bow(topic_water[1])

# and voila!
kullback_leibler(water_distribution, finance_distribution)

0.260778

You may notice that the distance for this is quite less, indicating a high similarity. This may be a bit off because of the small size of the corpus, where all topics are likely to contain a decent overlap of word probabilities. You will likely get a better value for a bigger corpus.

So, just remember, if you intend to use KL as a metric to measure similarity or distance between two distributions, avoid zeros by returning the ENTIRE distribution. Since it's unlikely any probability distribution will ever have absolute zeros for any feature/word, returning all the values like we did will make you good to go.

## So - what exactly are Distance Metrics? 

Having seen the practical usages of these measures (i.e, to find similarity), let's learn a little about what exactly Distance Measures and Metrics are. 

I mentioned in the previous section that KL was not a distance metric. There are 4 conditons for for a distance measure to be a matric:

1.	d(x,y) >= 0
2.  d(x,y) = 0 <=> x = y
3.  d(x,y) = d(y,x)
4.  d(x,z) <= d(x,y) + d(y,z)

That is: it must be non-negative; if x and y are the same, distance must be zero; it must be symmetric; and it must obey the triangle inequality law. 

Simple enough, right? 
Let's test these out for our measures.

In [21]:
# normal Hellinger
hellinger(water_distribution, finance_distribution)

0.25035161498501074

In [22]:
# we swap finance and water distributions and get the same value. It is indeed symmetric!
hellinger(finance_distribution, water_distribution)

0.25035161498501074

In [23]:
# if we pass the same values, it is zero.
hellinger(water_distribution, water_distribution)

0.0

In [24]:
# for triangle inequality let's use LDA document distributions
hellinger(lda_bow_finance, lda_bow_bank)

0.5167124677493228

In [25]:
# Triangle inequality works too!
hellinger(lda_bow_finance, lda_bow_water) + hellinger(lda_bow_water, lda_bow_bank)

0.591881815192646

So Hellinger is indeed a metric. Let's check out KL. 

In [26]:
kullback_leibler(finance_distribution, water_distribution)

0.25089684

In [27]:
kullback_leibler(water_distribution, finance_distribution)

0.260778

We immediately notice that when we swap the values they aren't equal! One of the four conditions not fitting is enough for it to not be a metric. 

However, just because it is not a metric, (strictly in the mathematical sense) does not mean that it is not useful to figure out the distance between two probability distributions. KL Divergence is widely used for this purpose, and is probably the most 'famous' distance measure in fields like Information Theory.

For a nice review of the mathematical differences between Hellinger and KL, [this](http://stats.stackexchange.com/questions/130432/differences-between-bhattacharyya-distance-and-kl-divergence) link does a very good job. 

## Conclusion

That brings us to the end of this small tutorial.
The scope for adding new similarity metrics is large, as there exist an even larger suite of metrics and methods to add to the matutils.py file. ([This](http://nzcsrsc08.canterbury.ac.nz/site/proceedings/Individual_Papers/pg049_Similarity_Measures_for_Text_Document_Clustering.pdf) is one paper which talks about some of them)

Looking forward to more PRs towards this functionality in Gensim! :)

In [28]:
from gensim import similarities

index = similarities.MatrixSimilarity(model[corpus])    ##创建索引
#print("corpus:",corpus)
#print("index:",index)
#print("lda_bow_finance:",lda_bow_finance)
sims = index[lda_bow_finance]
print("sims:",list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])

for doc_id, similarity in sims:
    print(texts[doc_id], similarity)

sims: [(0, 0.29681402), (1, 0.26814798), (2, 0.6278792), (3, 0.28040192), (4, 0.28666553), (5, 0.9995542), (6, 0.9997273), (7, 0.9890071), (8, 0.9996691), (9, 0.9969551), (10, 0.99989146)]
['bank', 'loan', 'sell'] 0.99989146
['bank', 'borrow', 'money'] 0.9997273
['finance', 'money', 'sell', 'bank'] 0.9996691
['money', 'transaction', 'bank', 'finance'] 0.9995542
['borrow', 'sell'] 0.9969551
['bank', 'finance'] 0.9890071
['bank', 'water', 'fall', 'flow'] 0.6278792
['bank', 'river', 'shore', 'water'] 0.29681402
['river', 'water', 'mud', 'tree'] 0.28666553
['bank', 'bank', 'water', 'rain', 'river'] 0.28040192
['river', 'water', 'flow', 'fast', 'tree'] 0.26814798


In [31]:
from gensim.summarization import summarize

ModuleNotFoundError: No module named 'gensim.summarization'

In [29]:
text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
　　Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.
　　The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn't think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley's sister, but they hadn't met for several years; in fact, Mrs. Dursley pretended she didn't have a sister, because her sister and her good-for-nothing husband were as unDursleyish as it was possible to be. The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street. The Dursleys knew that the Potters had a small son, too, but they had never even seen him. This boy was another good reason for keeping the Potters away; they didn't want Dudley mixing with a child like that.
　　When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday our story starts, there was nothing about the cloudy sky outside to suggest that strange and mysterious things would soon be happening all over the country. Mr. Dursley hummed as he picked out his most boring tie for work, and Mrs. Dursley gossiped away happily as she wrestled a screaming Dudley into his high chair.
　　None of them noticed a large, tawny owl flutter past the window.
　　At half past eight, Mr. Dursley picked up his briefcase, pecked Mrs. Dursley on the cheek, and tried to kiss Dudley good-bye but missed, because Dudley was now having a tantrum and throwing his cereal at the walls. "Little tyke," chortled Mr. Dursley as he left the house. He got into his car and backed out of number four's drive.
　　It was on the corner of the street that he noticed the first sign of something peculiar -a cat reading a map. For a second, Mr. Dursley didn't realize what he had seen -then he jerked his head around to look again. There was a tabby cat standing on the corner of Privet Drive, but there wasn't a map in sight. What could he have been thinking of? It must have been a trick of the light. Mr. Dursley blinked and stared at the cat. It stared back. As Mr. Dursley drove around the corner and up the road, he watched the cat in his mirror. It was now reading the sign that said Privet Drive -no, looking at the sign; cats couldn't read maps or signs. Mr. Dursley gave himself a little shake and put the cat out of his mind. As he drove toward town he thought of nothing except a large order of drills he was hoping to get that day.
　　But on the edge of town, drills were driven out of his mind by something else. As he sat in the usual morning traffic jam, he couldn't help noticing that there seemed to be a lot of strangely dressed people about. People in cloaks. Mr. Dursley couldn't bear people who dressed in funny clothes -the getups you saw on young people! He supposed this was some stupid new fashion. He drummed his fingers on the steering wheel and his eyes fell on a huddle of these weirdos standing quite close by. They were whispering excitedly together. Mr. Dursley was enraged to see that a couple of them weren't young at all; why, that man had to be older than he was, and wearing an emerald-green cloak! The nerve of him! But then it struck Mr. Dursley that this was probably some silly stunt -these people were obviously collecting for something... yes, that would be it. The traffic moved on and a few minutes later, Mr. Dursley arrived in the Grunnings parking lot, his mind back on drills.
"""
from gensim.summarization import summarize
print("1:",summarize(text))
print ("2:",summarize(text, word_count=50))



ModuleNotFoundError: No module named 'gensim.summarization'

In [ ]:
##关键词提取
from gensim.summarization import keywords
print (keywords(text))

In [30]:
from gensim.summarization import mz_keywords
mz_keywords(text)

ModuleNotFoundError: No module named 'gensim.summarization'